In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from sklearn.ensemble import GradientBoostingRegressor
import datetime
import category_encoders as ce
df = pd.read_csv('/Users/harleyhoffmann/dat-02-22/ClassMaterial/Unit3/data/bikeshare.csv')

In [68]:
#  - datetime: a timestamp collected hourly.
#  - season: a categorical column that lists the current season for that observation
# - holiday: a column (0 or 1), that detects whether or not it was a holiday
#  - workingday: a column (0 or 1), that encodes whether or not it was a workday or not
#  - weather: a categorical column that lists a light weather description for the observation
#  - temp: the temperature outside
#  - atemp: the temperature it feels like outside
#  - humidity: the humidity outside
#  - windspeed: the windspeed, in mph
#  - count: the number of bikes checked out during that hour

In [70]:
px.line(df, x='datetime', y='count', title='Bikeshare Rentals 2011-2012', labels={'datetime': 'Time', 'count': 'Number of Bikes/hour'})

No major surprises - big rentals increase in the summer months due to warm weather.
However, within those timelines there is a lot of noise, further discover is required.

In [83]:
px.line(df, x='datetime', y=['temp','atemp'], title='Temperature 2011-2012', labels={'datetime': 'Time'})

At first glance, it appears that the change in season, commonly associated with warmer temperatures in the summertime, has the largest impact on ride count. However we'll use our model fitting and scoring to determine which features actually have the most importance.

In [2]:
#it looks like the data is already sorted, but we're going to sort values just to be sure
df.sort_values(by=['datetime'], ascending=True, inplace=True)

In [3]:
#we're separating out the date aspects into their own columns to see how they effect the model
df['month'] = pd.DatetimeIndex(df['datetime']).month
df['year'] = pd.DatetimeIndex(df['datetime']).year
df['day_of_month'] = pd.DatetimeIndex(df['datetime']).day
df['hour_of_day'] = pd.DatetimeIndex(df['datetime']).hour


Let's take a look at the data now:

In [87]:
px.scatter(df, x='hour_of_day', y='count', title='Bikeshares through the day 2011-2012', labels={'hour_of_day': 'Hour of the day', 'count': 'Bike rentals'}, trendline='ols', color='count')

It looks like there is also great fluctuation in the bike rental counts based on the hour of the day - with more rentals throughout of the afternoon and evening.

In [4]:
# define a function that we can reuse to make a training, validation, and test set
def create_val_splits(df, val_units=15, return_val=False):
    """Function that will take in a dataset and split it up into training, validation, and test sets"""
    # split into training, validation, and test sets
    df = df.drop('datetime', axis=1)
    #we're dropping the dates because we moved them into new columns
    train = df.apply(lambda x: x.iloc[:-val_units]).reset_index(drop=True)
    test  = df.apply(lambda x: x.iloc[-val_units:]).reset_index(drop=True)
    
    if return_val:
        val   = train.apply(lambda x: x.iloc[-val_units:]).reset_index(drop=True)
        train = train.apply(lambda x: x.iloc[:-val_units]).reset_index(drop=True)
        return train, val, test
    else:
        return train, test

In [5]:
train, val, test = create_val_splits(df, val_units=2160, return_val=True)
#we're taking 2,160 rows of the most recent data for test and then the same again for the validation set
#2,160 rows is 90 days in hours
#24 hours in a day x 90 = 2,160

In [6]:
#creating our training and validation sets
X_train = train.drop('count', axis=1)
y_train = train['count']
X_val = val.drop('count', axis=1)
y_val = val['count']

In [7]:
#load in the GBM and the encoders so we can make a pipeline for each to test
from sklearn.pipeline import make_pipeline
gbm = GradientBoostingRegressor()
te = ce.TargetEncoder()
ore = ce.OrdinalEncoder()
ohe = ce.OneHotEncoder()

In [8]:
#make a pipeline to test each encoder to do our initial fitting
pipe1 = make_pipeline(te, gbm)
pipe2 = make_pipeline(ore, gbm)
pipe3 = make_pipeline(ohe, gbm)

In [24]:
#test each Encoder on the validation set using each pipeline
#Target Encoder
pipe1.fit(X_train, y_train)
pipe1.score(X_val,y_val)

/opt/anaconda3/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


0.6399675927742059

In [10]:
#Ordinal Encoder
pipe2.fit(X_train, y_train)
pipe2.score(X_val,y_val)

0.6462055953951504

In [11]:
#OneHotEncoder
pipe3.fit(X_train, y_train)
pipe3.score(X_val,y_val)

/opt/anaconda3/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


0.6491600460409128

After our initial fitting, we've got rather similar scores for each Encoder.
We're going to use the Target Encoder because we have categorical values in the weather and seasons columns and because we can take a look at the feature importances easier with this Encoder.

In [26]:
#Target Encoder
pipe1.fit(X_train, y_train)
pipe1.score(X_val,y_val)

/opt/anaconda3/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


0.6400880273415018

Take a look at the feature importances for each column to make assumptions about what affects the training data the most

In [27]:
#Load features and importance into a dataframe
feats = pd.DataFrame({'Features':X_train.columns, 'Importance':gbm.feature_importances_}).sort_values(by='Importance', ascending=False)
feats

,Features,Importance
11,hour_of_day,0.653688
2,workingday,0.101818
5,atemp,0.085911
4,temp,0.048691
8,month,0.038893
9,year,0.031475
6,humidity,0.020173
3,weather,0.015380
7,windspeed,0.001325
0,season,0.001324


Let's make some moving averages to try and improve our score.

In [50]:
#It appears hour_of_day has a big effect, so let's make some rolling 24, 48, 72 hour summary statistics
df['24_hour_avg']  = df['count'].rolling(24).mean().shift().values
df['48_hour_avg'] = df['count'].rolling(48).mean().shift().values
df['72_hour_avg'] = df['count'].rolling(72).mean().shift().values

In [89]:
#backfill, and split the data
df = df.bfill()

train, val, test = create_val_splits(df, val_units=2160, return_val=True)

In [59]:
#recreate our training x,y and validation x,y
X_train = train.drop('count', axis=1)
y_train = train['count']
X_val = val.drop('count', axis=1)
y_val = val['count']

In [66]:
#rerun our Target Encoder pipeline
pipe1.fit(X_train, y_train)
pipe1.score(X_val,y_val)

/opt/anaconda3/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning:

is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead



0.6566837244604828

Although only a modest improvement, these are still real improvements, so let's keep them.

In [73]:
# we'll first see how our model is overfitting
pipe1.score(X_train, y_train), pipe1.score(X_val, y_val)

(0.8693526953981537, 0.6566837244604828)

Quite a bit of overfitting, but we will tweak the parameters to mitigate this. 

In [78]:
# we'll try a GBM that only uses 70% of columns at each split
pipe4 = make_pipeline(ce.TargetEncoder(), GradientBoostingRegressor(max_features=0.7))

# and see our results
pipe4.fit(X_train, y_train)
pipe4.score(X_train, y_train), pipe4.score(X_val, y_val)

/opt/anaconda3/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning:

is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead



(0.8801733996338608, 0.6717333635156459)

With this tweak we've improved our out-of-sample score by a little bit more!

In [ ]:
#create last_hour, yesterday, last week

In [ ]:
#Now, we'll try adjusting our GBM parameters to see if we can improve our score.
#do steps 1-7 from Class 14 notebook

In [ ]:
#need to fit on the entire training set at the end before final test
#if the test score is less than validation, then that is overfitting